# EJERCICIO 1

In [1]:
import sqlite3 as sql
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [2]:
df = pd.read_csv('data/motor_data11-14lats.csv', sep = ',' )

#observaciones_nulas_antes = df[df['PREMIUM'].isnull()]
#print("Observaciones con valores nulos en la columna PREMIUM antes de la imputación:")
#observaciones_nulas_antes

In [3]:
#filas_con_nulos = df[df['SEATS_NUM'].isnull()]
#filas_con_nulos

In [4]:
#Crear columna Id
df['Id'] = range(1, len(df) + 1)

In [5]:
con = sql.connect("data/Base_db.sqlite") #esto crea una nueva base dentro de data y si ya existe solo la llama.
# cuando creo una coneccion estoy creando un objeto con metodos.

#activa el metodo cursor que permite ejecutar codigo de sql
cursor = con.cursor()
#con.close()

In [6]:
# subir el DataFrame en SQL
df.to_sql('Seguros_de_autos', con, if_exists='replace', index=False)

293537

In [7]:
#consultar las tablas que tenemos
cursor.execute(
    '''
    SELECT tbl_name
    FROM sqlite_master
    WHERE type = 'table'
    '''
)

#cursor.fetchall() lo que hace es consultar las tablas que tenemos
print(cursor.fetchall())

[('tempora_gru1_prom',), ('Seguros_de_autos',)]


In [8]:
# Crear datos nulos en la columna PREMIUM para filas con un 9 en su Id
cursor.execute("""
--Seleccionar la tabla Seguros_de_autos para modifica
UPDATE Seguros_de_autos
SET PREMIUM = CASE
    WHEN Id LIKE '%9%' THEN NULL
    ELSE PREMIUM
END

""")
con.commit()

In [9]:
df


,SEX,INSR_BEGIN,INSR_END,EFFECTIVE_YR,INSR_TYPE,INSURED_VALUE,PREMIUM,OBJECT_ID,PROD_YEAR,SEATS_NUM,CARRYING_CAPACITY,TYPE_VEHICLE,CCM_TON,MAKE,USAGE,CLAIM_PAID,Id
0,0,08-AUG-13,07-AUG-14,08,1202,519755.22,7209.140,5000029885,2007.0,4.0,6.0,Pick-up,3153.0,NISSAN,Own Goods,NaN,1
1,0,08-AUG-12,07-AUG-13,08,1202,519755.22,7203.890,5000029885,2007.0,4.0,6.0,Pick-up,3153.0,NISSAN,Own Goods,NaN,2
2,0,08-AUG-11,07-AUG-12,08,1202,519755.22,7045.804,5000029885,2007.0,4.0,6.0,Pick-up,3153.0,NISSAN,Own Goods,NaN,3
3,0,08-JUL-11,07-AUG-11,08,1202,519755.22,287.250,5000029885,2007.0,4.0,6.0,Pick-up,3153.0,NISSAN,Own Goods,NaN,4
4,0,08-AUG-13,07-AUG-14,08,1202,285451.24,4286.900,5000029901,2010.0,4.0,7.0,Pick-up,2494.0,TOYOTA,Own Goods,19894.43,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293532,2,11-AUG-12,10-AUG-13,88,1201,0.00,577.700,5000047311,1985.0,4.0,NaN,Automobile,1295.0,TOYOTA,Private,NaN,293533
293533,2,11-AUG-11,10-AUG-12,88,1201,0.00,695.318,5000047311,1985.0,4.0,NaN,Automobile,1295.0,TOYOTA,Private,NaN,293534
293534,2,01-JUN-13,31-MAY-14,13,1201,250000.00,4778.450,5000949222,1993.0,4.0,NaN,Automobile,1295.0,TOYOTA,Private,NaN,293535
293535,0,08-JUL-12,07-JUL-13,85,1202,0.00,1057.427,5000049231,2017.0,2.0,0.0,Truck,4570.0,ISUZU,General Cartage,NaN,293536


In [10]:
#Agregar columna rasgo_asegurado en la tabla Seguros_de_autos
cursor.execute("""
ALTER TABLE Seguros_de_autos
ADD COLUMN rango_asegurado VARCHAR(50);

""")

In [11]:
# Discretice la variable INSURED_VALUE
cursor.execute("""
--Seleccionar la tabla Seguros_de_autos para modifica
UPDATE Seguros_de_autos
SET rango_asegurado = CASE
    WHEN INSURED_VALUE < 1000 THEN '1e3'
    WHEN INSURED_VALUE >= 1000 AND INSURED_VALUE < 1000000 THEN '1e6'
    WHEN INSURED_VALUE >= 1000000 AND INSURED_VALUE < 1000000000 THEN '1e9'
    ELSE '1e12'
END;

""")
con.commit()

# Agrupación Grupo 1

### Promedio

In [12]:
#Crea tabla temporal del los posibles grupos de las combinaciones de las columas con el promedio
cursor.execute("""
-- Tabla temporal para guardar el promedio de la agrupación
CREATE TEMPORARY TABLE agru_prom AS
SELECT
    SEX,
    PROD_YEAR,
    SEATS_NUM,
    rango_asegurado,
    AVG(PREMIUM) AS prom_PREMIUM 
    -- Guarda los valores en esta columna de los promedio por grupo
    -- Es el promedio para cada grupo de combi de columnas
FROM
    Seguros_de_autos
--WHERE
    --PREMIUM IS NOT NULL
    
--Agrupa las filas de la tabla según las columnas especificadas   
--Es decir, va a calcular el promedio PREMIUM dentro de cada grupo
GROUP BY           
    SEX, PROD_YEAR, SEATS_NUM, rango_asegurado;
""")
con.commit()

In [13]:
#Crear una tabla temporal copia de seguros de autos para guardar la tabla con los datos imputados con con la agrupación y estadística especificada
cursor.execute("""
CREATE TEMPORARY TABLE tempora_gru1_prom AS
SELECT *
FROM Seguros_de_autos;
""")



In [14]:

cursor.execute("""
UPDATE tempora_gru1_prom
SET PREMIUM = (
    SELECT agru_prom.prom_PREMIUM
    FROM agru_prom
    WHERE
        tempora_gru1_prom.SEX = agru_prom.SEX AND
        tempora_gru1_prom.PROD_YEAR = agru_prom.PROD_YEAR AND
        tempora_gru1_prom.SEATS_NUM = agru_prom.SEATS_NUM AND
        tempora_gru1_prom.rango_asegurado = agru_prom.rango_asegurado
)
WHERE tempora_gru1_prom.PREMIUM IS NULL;
""")

con.commit()


In [15]:
#cursor.execute("""
#DROP TABLE tabla_agrupa
#""")

### Máximo

In [16]:
#Crea tabla temporal del los posibles grupos de las combinaciones de las columas
cursor.execute("""
-- Tabla temporal para guardar el promedio de la agrupación
CREATE TEMPORARY TABLE agru_max AS
SELECT
    SEX,
    PROD_YEAR,
    SEATS_NUM,
    rango_asegurado,
    MAX(PREMIUM) AS max_PREMIUM 
    -- Guarda los valores en esta columna deL máximo por grupo
    -- Es el max para cada grupo de combi de columnas
FROM
    Seguros_de_autos
WHERE
    PREMIUM IS NOT NULL
    
--Agrupa las filas de la tabla según las columnas especificadas   
--Es decir, va a calcular el max PREMIUM dentro de cada grupo
GROUP BY           
    SEX, PROD_YEAR, SEATS_NUM, rango_asegurado;
""")
con.commit()

In [17]:
#Crear una tabla temporal copia de seguros de autos para guardar la tabla con los datos imputados con con la agrupación y estadística especificada
cursor.execute("""
CREATE TEMPORARY TABLE tempora_gru1_max AS
SELECT *
FROM Seguros_de_autos;
""")

In [18]:
#modificar los nulos con el maximo
cursor.execute("""
UPDATE tempora_gru1_max
SET PREMIUM = (
    SELECT agru_max.max_PREMIUM
    FROM agru_max
    WHERE
        tempora_gru1_max.SEX = agru_max.SEX AND
        tempora_gru1_max.PROD_YEAR = agru_max.PROD_YEAR AND
        tempora_gru1_max.SEATS_NUM = agru_max.SEATS_NUM AND
        tempora_gru1_max.rango_asegurado = agru_max.rango_asegurado
)
WHERE tempora_gru1_max.PREMIUM IS NULL;
""")
con.commit()

In [19]:
Seguros_de_autos = pd.read_sql(
    '''
    SELECT *
    FROM tempora_gru1_max
    ''',
    con
)
df2 = pd.DataFrame(Seguros_de_autos)
df_nulos = df2[df2['PREMIUM'].isnull()]
df_nulos

,SEX,INSR_BEGIN,INSR_END,EFFECTIVE_YR,INSR_TYPE,INSURED_VALUE,PREMIUM,OBJECT_ID,PROD_YEAR,SEATS_NUM,CARRYING_CAPACITY,TYPE_VEHICLE,CCM_TON,MAKE,USAGE,CLAIM_PAID,Id,rango_asegurado
956,1,28-AUG-12,27-AUG-13,08,1201,72345.93,NaN,5000029641,1971.0,4.0,0.0,Automobile,1648.0,PEUGEOT,Taxi,NaN,957,1e6
957,2,07-JAN-14,06-JAN-15,14,1202,0.00,NaN,5000029957,1972.0,2.0,57.3,Truck,5184.0,FIAT,General Cartage,NaN,958,1e3
958,2,24-JUL-12,23-JUL-13,05,1202,0.00,NaN,5000029957,1972.0,2.0,57.3,Truck,5184.0,FIAT,General Cartage,NaN,959,1e3
959,2,24-JUL-11,23-JUL-12,05,1202,0.00,NaN,5000029957,1972.0,2.0,57.3,Truck,5184.0,FIAT,General Cartage,NaN,960,1e3
1895,0,12-SEP-11,03-MAY-12,50,1202,0.00,NaN,5000027916,2010.0,30.0,32.0,Bus,0.0,TOYOTA,Own service,NaN,1896,1e3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293518,2,10-JAN-14,09-JAN-15,12,1201,600000.00,NaN,5000846035,2002.0,7.0,NaN,Station Wagones,2779.0,TOYOTA,Private,NaN,293519,1e6
293519,2,08-JAN-12,07-JAN-13,12,1201,600000.00,NaN,5000846035,2002.0,7.0,NaN,Station Wagones,2779.0,TOYOTA,Private,NaN,293520,1e6
293520,2,10-JAN-13,09-JAN-14,12,1201,600000.00,NaN,5000846035,2002.0,7.0,NaN,Station Wagones,2779.0,TOYOTA,Private,NaN,293521,1e6
293535,0,08-JUL-12,07-JUL-13,85,1202,0.00,NaN,5000049231,2017.0,2.0,0.0,Truck,4570.0,ISUZU,General Cartage,NaN,293536,1e3


# Imputación Grupo 2: PROD_YEAR, SEATS_NUM, TYPE_VEHICLE y rango_asegurado

### Promedio 

In [20]:
cursor.execute("""
-- Tabla temporal para guardar el promedio de la agrupación
CREATE TEMPORARY TABLE tabla_agrupa_prom AS
SELECT
    PROD_YEAR,
    SEATS_NUM,
    TYPE_VEHICLE,
    rango_asegurado,
    AVG(PREMIUM) AS imput_PREMIUM_prom 
FROM
    Seguros_de_autos
WHERE
    PREMIUM IS NOT NULL
    
--Agrupa las filas de la tabla según las columnas especificadas   
--Es decir, va a calcular el promedio PREMIUM dentro de cada grupo
GROUP BY           
    PROD_YEAR, SEATS_NUM, TYPE_VEHICLE, rango_asegurado;
""")
con.commit()

In [21]:
#Crear una tabla temporal copia de seguros de autos para guardar la tabla con los datos imputados con con la agrupación y estadística especificada
cursor.execute("""
CREATE TEMPORARY TABLE tempora_gru2_prom AS
SELECT *
FROM Seguros_de_autos;
""")

In [22]:
#modificar los nulos con el promedio
cursor.execute("""
UPDATE tempora_gru2_prom
SET PREMIUM = (
    SELECT tabla_agrupa_prom.imput_PREMIUM_prom
    FROM tabla_agrupa_prom
    -- Busca que las columnas de cada observación coincida con la de al agrupación y así asignarle el valor.
    WHERE
        tempora_gru2_prom.PROD_YEAR = tabla_agrupa_prom.PROD_YEAR AND
        tempora_gru2_prom.SEATS_NUM = tabla_agrupa_prom.SEATS_NUM AND
        tempora_gru2_prom.rango_asegurado = tabla_agrupa_prom.rango_asegurado AND
        tempora_gru2_prom.TYPE_VEHICLE = tabla_agrupa_prom.TYPE_VEHICLE
)
WHERE tempora_gru2_prom.PREMIUM IS NULL;
""")

### Máximo

In [23]:
#Crea tabla temporal del los posibles grupos de las combinaciones de las columas con el promedio
cursor.execute("""
-- Tabla temporal para guardar el promedio de la agrupación
CREATE TEMPORARY TABLE tabla_agrupa_max AS
SELECT
    PROD_YEAR,
    SEATS_NUM,
    TYPE_VEHICLE,
    rango_asegurado,
    MAX(PREMIUM) AS imput_PREMIUM_max
FROM
    Seguros_de_autos
WHERE
    PREMIUM IS NOT NULL
    
--Agrupa las filas de la tabla según las columnas especificadas   
--Es decir, va a calcular el promedio PREMIUM dentro de cada grupo
GROUP BY           
    PROD_YEAR, SEATS_NUM, TYPE_VEHICLE, rango_asegurado;
""")
con.commit()


In [24]:
#Crear una tabla temporal copia de seguros de autos para guardar la tabla con los datos imputados con con la agrupación y estadística especificada
cursor.execute("""
CREATE TEMPORARY TABLE tempora_gru2_max AS
SELECT *
FROM Seguros_de_autos;
""")

In [25]:
#modificar los nulos con el maximo
cursor.execute("""
UPDATE tempora_gru2_max
SET PREMIUM = (
    SELECT tabla_agrupa_max.imput_PREMIUM_max
    FROM tabla_agrupa_max
    WHERE
        tempora_gru2_max.PROD_YEAR = tabla_agrupa_max.PROD_YEAR AND
        tempora_gru2_max.SEATS_NUM = tabla_agrupa_max.SEATS_NUM AND
        tempora_gru2_max.rango_asegurado = tabla_agrupa_max.rango_asegurado AND
        tempora_gru2_max.TYPE_VEHICLE = tabla_agrupa_max.TYPE_VEHICLE
)
WHERE tempora_gru2_max.PREMIUM IS NULL;
""")


### Extraer las tablas de SQL a DataFrames en python

In [26]:
# SQL a python
Seguros_de_autos = pd.read_sql(
    '''
    SELECT *
    FROM Seguros_de_autos
    ''',
    con
)

df_grup1_prom = pd.read_sql(
    '''
    SELECT *
    FROM tempora_gru1_prom
    ''',
    con
)

df_grup1_max = pd.read_sql(
    '''
    SELECT *
    FROM tempora_gru1_max
    ''',
    con
)

df_grup2_prom = pd.read_sql(
    '''
    SELECT *
    FROM tempora_gru2_prom
    ''',
    con
)

df_grup2_max = pd.read_sql(
    '''
    SELECT *
    FROM tempora_gru2_max
    ''',
    con
)

#me traigo de SQL todas las tablas temporales a df a python

In [27]:
#Convierto las tablas en dataframes de pandas
Seguros_de_autos = pd.DataFrame(Seguros_de_autos)
df_grup1_prom = pd.DataFrame(df_grup1_prom)
df_grup1_max = pd.DataFrame(df_grup1_max)
df_grup2_prom = pd.DataFrame(df_grup2_prom)
df_grup2_max = pd.DataFrame(df_grup2_max)



# Métricas

## Grupo 1:

### Promedio

In [28]:
# Valores originales
y_true = np.array(df['PREMIUM']) #Se extrae la columna PREMIUM DEL df original

# Valores imputados
y_pred = np.array(df_grup1_prom['PREMIUM'])

# Enmascarar los valores no nulos https://numpy.org/doc/stable/reference/generated/numpy.isnan.html
# Esta "mascara" solo las considera cuando ambas columnas no tiene nulo en la observación, por eso el ~
mask = ~np.isnan(y_true) & ~np.isnan(y_pred)

# Filtrar los valores no nulos
y_true_filtrado = y_true[mask]
y_pred_filtrado = y_pred[mask]

# Calcular RSS
rss = np.sum((y_true_filtrado - y_pred_filtrado)**2)
print(f"RSS: {rss}")

# Calcular RMSE
rmse = np.sqrt(mean_squared_error(y_true_filtrado, y_pred_filtrado))
print(f"RMSE: {rmse}")

# Calcular R²
r2 = r2_score(y_true_filtrado, y_pred_filtrado)
print(f"R²: {r2}")


RSS: 3435222762469.8433
RMSE: 3431.8111606909
R²: 0.8557281264858414


El RMSE es la raíz cuadrada del error cuadrático medio y proporciona una medida de la magnitud promedio de los errores de predicción, y nótese que supera los 3 billones, es decir hubo una discrepancia abismal y este valor es similar o superior en las demás metricas. No obstante, en este caso R² muestra una relativa cercanía a 1, lo que permite intuir que tomar el promedio no es una mala aproximación (comparada con otras).

### Máximo

In [29]:

y_pred = np.array(df_grup1_max['PREMIUM'])

mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
y_true_filtrado = y_true[mask]
y_pred_filtrado = y_pred[mask]

rss = np.sum((y_true_filtrado - y_pred_filtrado)**2)
print(f"RSS: {rss}")

rmse = np.sqrt(mean_squared_error(y_true_filtrado, y_pred_filtrado))
print(f"RMSE: {rmse}")

r2 = r2_score(y_true_filtrado, y_pred_filtrado)
print(f"R²: {r2}")


RSS: 98466991228570.58
RMSE: 18373.473349252374
R²: -3.1353991534552783


Nótese que la métrica R² al considerara el máximo da negativo, en teoría el resultada debería estar entre cero y uno, donde uno implica la mejor predicción, no obstante también es posible que de negativa, lo que quiere decir que la predicción y mucho más mala. Y considero que el resultado es coherente, pues el máximo es extremo, lo que se aleja de la realidad en este caso que son valores tan diversos.

## Grupo 2:

### Promedio

In [30]:
y_pred = np.array(df_grup2_prom['PREMIUM'])

mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
y_true_filtrado = y_true[mask]
y_pred_filtrado = y_pred[mask]

rss = np.sum((y_true_filtrado - y_pred_filtrado)**2)
print(f"RSS: {rss}")

rmse = np.sqrt(mean_squared_error(y_true_filtrado, y_pred_filtrado))
print(f"RMSE: {rmse}")

r2 = r2_score(y_true_filtrado, y_pred_filtrado)
print(f"R²: {r2}")


RSS: 3705084352233.7363
RMSE: 3566.1510734461062
R²: 0.8442776314738616


Con esta agrupación se muestra un valor R² inferior a calulado en el promedio del grupo 1, esto implica que esta escogencia de columnas para la agrupación no es la mejor para realizar la imputación, ya que estas columnas no son tan decisivas para la prima, comparadas con las del grupo 1. Y en las otras métricas muestra valores superiores lo que igualmente evidencia lo antes mencionado.

### Máximo

In [31]:
y_pred = np.array(df_grup2_max['PREMIUM'])

mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
y_true_filtrado = y_true[mask]
y_pred_filtrado = y_pred[mask]

rss = np.sum((y_true_filtrado - y_pred_filtrado)**2)
print(f"RSS: {rss}")

rmse = np.sqrt(mean_squared_error(y_true_filtrado, y_pred_filtrado))
print(f"RMSE: {rmse}")

r2 = r2_score(y_true_filtrado, y_pred_filtrado)
print(f"R²: {r2}")


RSS: 50831680906657.234
RMSE: 13208.938648275243
R²: -1.13642362613886


Curiosamente considerando el max con esta agrupación el valor de R² es mayor que la del grupo 1, por lo tanto se concluye que para la estadística de máximo es mejor esta agrupación de columnas y para el promedio es mejor la agrupación del grupo 1.





# EJERCICIO 2

In [32]:
import numpy as np

def kernel_sigmoid(z):
    '''
    Función signomide o función logística, la cual tiene dominio en los 
    números reales e imágenes entre cero y uno.

    Parameters
    ----------
    z (int): Cualqueir número real.
    
    Returns
    ------    
    (int): Número entre cero y uno.
    '''    
    return 1 / (1 + np.exp(-z))


In [33]:
def forward_propagation(W, b, X, y):
    """
    Propagación hacia adelante, es dericr, calcula las predicciones del modelo y el costo asociado para un 
    conjunto dado de datos de entrada, utilizando los parámetros indicados W como peso y b como el sesgo.

    Parameters:
    -----------
    W (numpy.ndarray) : Vector de pesos de la regresión logística.
        
    b (float) : Sesgo de la regresión logística.
        
    X (numpy.ndarray): 
        Parámetro con la matriz de la variables caracteristicas. Cada fila representa una observacion, 
        y cada columna representa una variable (característica), es utilizado para trianing.
        
    y (numpy.ndarray) : Array de numpy (columna) con las estiquestas reales de los datos.

    Returns:
    --------
    A (numpy.ndarray): Array de numpy con las predicciones del modelo. Cada elemento del array representa la probabilidad
    de que el ejemplo correspondiente pertenezca a la clase positiva (una de las dos etiquetas reales).

        
    cost (float) : Valor de la función de costo calculado.
    """  
    
    #obtener el número de filas de los datos de training
    data_number = X.shape[0]
    # Calcula para cada entrada de X un combinación linea con los parámetros de la regresión W y b
    Z = np.dot(W, X.T) + b
    # Aplicar la función sigmoide a cada valor de z para obtener las predicciones
    A = kernel_sigmoid(Z)
    # Calculo de la funcion de costos de la regresion. Sirve para medir que tan equivocado estás
    cost = (- 1 / data_number) * np.sum(y * np.log(A) + (1 - y) * (np.log(1 - A)))
    #Retorna las predicciones del modelo y el costo asociado con esas predicciones. 
    return A, cost

In [34]:
def backward_propagation(X, A, y):
    """
    Se encargar de culcular los gredientes de parámetro W (peso) y b (sesgo), pasa así optimizar el costo del modelo.

    Parameters:
    -----------
    X (numpy.ndarray): Parámetro con la matriz de la variables caracteristicas. Cada fila representa una observacion, 
        y cada columna representa una variable (característica).
    A (numpy.ndarray): Valores predichos de la propagación hacia adelante.
    y (numpy.ndarray): Array de numpy (columna) con las estiquestas reales de los datos.

    Returns:
    --------
    dW (numpy.ndarray): Gradiente de la matriz de pesos.
    db (float): Gradiente del sesgo.
    """
    
    #obtener el número de filas de los datos de training
    data_number = X.shape[0]
    # los gradites indica la dirección y la magnitud en la que debemos ajustar los parámetros
    #Calcula el gradiente del array W (pesos) para minimizar el costo
    dW = (1 / data_number) * np.dot((A - y), X)
    #Calcula el gradiente del array b (sesgo) para minimizar el costo
    db = (1 / data_number) * np.sum(A - y)
    #Devuelve los gradites respectivos
    return dW, db


In [35]:
def optimize(W, b, X, y, num_iterations, learning_rate):
    """
    El método optimize ajusta iterativamente los parámetros del modelo utilizando el gradiente 
    descendente para minimizar el costo. Guarda los costos cada 100 iteraciones permite monitorear
    el proceso de entrenamiento y asegurarse de que el modelo está aprendiendo correctamente.
    
    Parameters:
    -----------
    W (numpy.ndarray): Vector de pesos de la regresión logística.
    b (float): Valor de sesgo.
    X (numpy.ndarray): Parámetro con la matriz de la variables caracteristicas. Cada fila representa una observacion, 
                       y cada columna representa una variable (característica).
    y (numpy.ndarray): Array de numpy (columna) con las estiquestas reales de los datos.
    num_iterations (int): Número de iteraciones para el bucle de optimización.
    learning_rate (float): Tasa de aprendizaje para el descenso de gradiente.

    Returns:
    --------
    params (dict): Diccionario que contiene los W (pesos) y el b (sesgo) optimizados.
    gradients (dict): Diccionario que contiene los gradientes finales de los pesos y el sesgo.
    costs (list): Lista de costos registrados durante el proceso de optimización.
    """
    #crea lista para guardar los costos.
    costs = []

    # For para ejecutar las actualizaciones segun el número de iteraciones especificado
    for i in range(num_iterations):
        #se ejecuta el metodo de propagacion hacia adelante con los parámetros ingresados
        A, cost = forward_propagation(W, b, X, y)
        #se ejecuta el metodo de propagacion hacia atras con los parámetros ingresados
        dW, db = backward_propagation(X, A, y)
        #Se modifica W con la tasa de aprendizaje indicada y el gradiente de W
        W = W - learning_rate * dW
        #Se modifica b con la tasa de aprendizaje indicada y el gradiente de b
        b = b - learning_rate * db
        #si la iteración es módulo 100 se guarda en la lista costs definida fura del for
        #Esto para ver con se va comportando los costos y si van disminuyendo o si se estancó.
        if i % 100 == 0:
            costs.append(cost)

    #Diccionario para guardar los valores finales de los parámetros
    params = {
        "W": W, 
        "b": b
    }
    #Diccionario para guardar los valores finales de los gradientes
    gradients = {
        "dW": dW,
        "db": db
    }
    #Devolver los diccionarios de los parámetros, gradientes y la lista de la evolución de los costos.
    return params, gradients, costs

In [36]:
def predict(W, b, X):
    """
    Este método realiza predicciones utilizando los pesos y el sesgo que aprendió.

    Parameters:
    -----------
    W (numpy.ndarray): Vector de pesos de la regresión logística.
    b (float): Valor de sesgo.
    X (numpy.ndarray): Parámetro con la matriz de la variables caracteristicas. Cada fila representa una observacion, 
                       y cada columna representa una variable (característica)

    Returns:
    --------
    y_prediction (numpy.ndarray): Etiquetas predichas (valores 0 o 1).
    """   
    #obtener el número de filas de los datos de training
    data_number = X.shape[0]
     #Inicializa un array a cero donde van a estar la predicciónes.
    y_prediction = np.zeros((1, data_number))
    # Calcula para cada entrada de X un combinación linea con los parámetros de la regresión W y b
    Z = np.dot(W, X.T) + b
    #aplica la función sigmoide en Z calculado anteriormente, obteniendo así las "activaciones" para cada entrada.
    A = kernel_sigmoid(Z)
    #for que recorre las activaciones de Z para generar las respectivas predicciones
    for i in range(A.shape[1]):
        #Es 1 cuando la activación es mayor que 0.5, sino se le asígna 0.
        y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
        
    #Retorna todas las predicciones
    return y_prediction

In [37]:
def model_regresion_logistico(X_train, y_train, X_val, y_val, num_iterations=2000, learning_rate=0.5):
    """
    Método que entrena un modelo de regresión logística y lo evalúa en los datos de entrenamiento y validación.

    Parameters:
    -----------
    X_train (numpy.ndarray): Partición de características seleccionadas para entrenamiento.
    y_train (numpy.ndarray): Partición para entrenamiento.
    X_val (numpy.ndarray): Partición de características para validación (testing).
    y_val (numpy.ndarray): Particioón para validación (testing).
    num_iterations (int): Número de iteraciones para el entrenamiento.
    learning_rate (float): Tasa de aprendizaje para el descenso de gradiente.

    Returns:
    --------
    resultados (dict): Diccionario que contiene la precisión en entrenamiento, la precisión en validación y el historial de los costos.
    """
    
    # Determinar el número de características
    dimensions = X_train.shape[1]
    # Inicializar los pesos y el sesgo a cero
    W = np.zeros(shape=(1, dimensions))
    b = 0
    
    # Optimizar los parámetros W y b utilizando los datos de entrenamiento
    params, gradients, costs = optimize(W, b, X_train, y_train, num_iterations, learning_rate)
    
    # Obtener los valores optimizados de W y b
    W = params["W"]
    b = params["b"]
    
    # Realizar predicciones en los datos de entrenamiento y validación
    y_prediction_train = predict(W, b, X_train)
    y_prediction_validation = predict(W, b, X_val)
    
    # Calcular el ajuste (precisión) del modelo en los datos de entrenamiento y validación
    ajuste_entrenamiento = 100 - np.mean(np.abs(y_prediction_train - y_train)) * 100
    ajuste_val = 100 - np.mean(np.abs(y_prediction_validation - y_val)) * 100
    
    # Crear un diccionario con los resultados
    resultados = {
        "Ajuste entrenamiento": ajuste_entrenamiento,
        "Ajuste testeo": ajuste_val,
        "Costo en proceso": costs
    }
    
    # Retornar el diccionario con los resultados
    return resultados

## Ejecución de las 10 corridas del código

### Código Original

In [38]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time

def kernel_sigmoid(z):
    return 1 / (1 + np.exp(-z))

def forward_propagation(W, b, X, y):
    data_number = X.shape[0]
    Z = np.dot(W, X.T) + b
    A = kernel_sigmoid(Z)
    cost = (-1 / data_number) * np.sum(y * np.log(A) + (1 - y) * (np.log(1 - A)))
    return A, cost

def backward_propagation(X, A, y):
    data_number = X.shape[0]
    dW = (1 / data_number) * np.dot((A - y), X)
    db = (1 / data_number) * np.sum(A - y)
    return dW, db

def optimize(W, b, X, y, num_iterations, learning_rate):
    costs = []
    for i in range(num_iterations):
        A, cost = forward_propagation(W, b, X, y)
        dW, db = backward_propagation(X, A, y)
        W = W - learning_rate * dW
        b = b - learning_rate * db
        if i % 100 == 0:
            costs.append(cost)
    params = {"W": W, "b": b}
    gradients = {"dW": dW, "db": db}
    return params, gradients, costs

def predict(W, b, X):
    Z = np.dot(W, X.T) + b
    A = kernel_sigmoid(Z)
    y_prediction = (A > 0.5).astype(int)
    return y_prediction

def model_regresion_logistico1(X_train, y_train, X_val, y_val, num_iterations=2000, learning_rate=0.5):
    dimensions = X_train.shape[1]
    W = np.zeros((1, dimensions))
    b = 0
    params, gradients, costs = optimize(W, b, X_train, y_train, num_iterations, learning_rate)
    W = params["W"]
    b = params["b"]
    
    y_prediction_train = predict(W, b, X_train)
    y_prediction_validation = predict(W, b, X_val)
    
    lista = {
        "Ajuste entrenamiento": 100 - np.mean(np.abs(y_prediction_train - y_train)) * 100,
        "Ajuste testeo": 100 - np.mean(np.abs(y_prediction_validation - y_val)) * 100,
        "Costo en proceso": costs
    }
    return lista

# Datos y las transformaciones
datos = pd.read_csv("Diabetes.txt")
X = np.array(datos.drop(["Outcome"], axis=1))
y = np.array(datos["Outcome"])
X = (X - np.min(X)) / (np.max(X) - np.min(X))

# Separa la muestra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=None)

# Medir el tiempo de las 10 ejecuciones
tiempos_ori = []
for i in range(10):
    inicio = time.time()
    model_regresion_logistico1(X_train, y_train, X_val, y_val, num_iterations=1000, learning_rate=0.003)
    fin = time.time()
    tiempos_ori.append((i + 1, fin - inicio))

tiempos_original = pd.DataFrame({
    "Run": [tiempo[0] for tiempo in tiempos_ori],
    "Original": [tiempo[1] for tiempo in tiempos_ori]
})

tiempos_original

,Run,Original
0,1,0.052002
1,2,0.049057
2,3,0.048943
3,4,0.053001
4,5,0.045058
5,6,0.045005
6,7,0.054937
7,8,0.047000
8,9,0.056001
9,10,0.054001


### Optimizado

In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def model_regresion_logistico2(X_train, y_train, X_val, y_val, num_iterations=1000, learning_rate=0.01, batch_size=64):
    dimensions = X_train.shape[1]
    W = np.zeros(dimensions)
    b = 0
    data_number = X_train.shape[0]
    
    costs = []
    for i in range(num_iterations):
        Z = np.dot(X_train, W) + b
        A = sigmoid(Z)
        
        cost = (-1 / data_number) * np.sum(y_train * np.log(A) + (1 - y_train) * np.log(1 - A))
        
        dW = (1 / data_number) * np.dot(X_train.T, (A - y_train))
        db = (1 / data_number) * np.sum(A - y_train)
        
        W -= learning_rate * dW
        b -= learning_rate * db
        
        if i % 100 == 0:
            costs.append(cost)

    #precalcalcular parámetros de la funcion
    Z_train = np.dot(X_train, W) + b
    Z_val = np.dot(X_val, W) + b
    #Y luego vectorizar la función predicción evitando un ciclo for
    y_prediction_train = sigmoid(Z_train) >= 0.5
    y_prediction_validation = sigmoid(Z_val) >= 0.5
    
    #No usar diccionario y ejecutar las operaciones y resultados en el return
    return 100 - np.mean(np.abs(y_prediction_train - y_train)) * 100, 100 - np.mean(np.abs(y_prediction_validation - y_val)) * 100, costs

# Datos y las transformaciones
datos = pd.read_csv("Diabetes.txt")
X = np.array(datos.drop(["Outcome"], axis=1))
y = np.array(datos["Outcome"])
X = (X - np.min(X, axis=0)) / (np.max(X, axis=0) - np.min(X, axis=0))

# Separa la muestra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=None)

# Medir el tiempo de las 10 ejecuciones
tiempos_opti = []
for i in range(10):
    inicio = time.time()
    ajuste_entrenamiento, ajuste_testeo, costs = model_regresion_logistico2(X_train, y_train, X_val, y_val, num_iterations=1000, learning_rate=0.01)
    fin = time.time()
    tiempos_opti.append((i + 1, fin - inicio))

tiempos_optimizado = pd.DataFrame({
    "Run": [tiempo[0] for tiempo in tiempos_opti],
    "Optimizado": [tiempo[1] for tiempo in tiempos_opti]
})

tiempos_optimizado

,Run,Optimizado
0,1,0.047053
1,2,0.049573
2,3,0.052001
3,4,0.042033
4,5,0.041085
5,6,0.039143
6,7,0.041139
7,8,0.048169
8,9,0.042047
9,10,0.043545


### Sklearn

In [40]:
from sklearn.linear_model import LogisticRegression


# Datos y transformaciones
datos = pd.read_csv("Diabetes.txt")
X = np.array(datos.drop(["Outcome"], axis=1))
y = np.array(datos["Outcome"])
X = (X - np.min(X)) / (np.max(X) - np.min(X))

# Separa la muestra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

# Medir tiempo del modelo LogisticRegression en 10 ejecuciones
tiempos_sklearn = []
for i in range(10):
    inicio = time.time()
    modelo = LogisticRegression(max_iter=1000)
    modelo.fit(X_train, y_train)
    fin = time.time()
    tiempos_sklearn.append((i + 1, fin - inicio))

# Resultados
tiempos_sklearn = pd.DataFrame({
    "Run": [tiempo[0] for tiempo in tiempos_sklearn],
    "Sklearn": [tiempo[1] for tiempo in tiempos_sklearn]
})

tiempos_sklearn


,Run,Sklearn
0,1,0.003000
1,2,0.003001
2,3,0.003000
3,4,0.001999
4,5,0.003000
5,6,0.002998
6,7,0.002002
7,8,0.003000
8,9,0.003000
9,10,0.002000


### Tabla Resumen

In [48]:
# Extraer columnas de cada df de los tiempos de ejecución
df1 = tiempos_original[['Run', 'Original']]
df2 = tiempos_optimizado[['Optimizado']]
df3 = tiempos_sklearn[['Sklearn']]

# Concatenar las columnas extraídas una al lado de la otra
df_colums = pd.concat([df1, df2, df3], axis=1)

# Crear observación promedio
fila_prom = pd.DataFrame({
    'Run': ['Media'],
    'Original': [df1['Original'].mean()],
    'Optimizado': [df2['Optimizado'].mean()],
    'Sklearn': [df3['Sklearn'].mean()]
})

# Concatenar la fila de promedios al DataFrame
df_resumen = pd.concat([df_colums, fila_prom], axis=0, ignore_index=True)

df_resumen

,Run,Original,Optimizado,Sklearn
0,1,0.052002,0.047053,0.003000
1,2,0.049057,0.049573,0.003001
2,3,0.048943,0.052001,0.003000
3,4,0.053001,0.042033,0.001999
4,5,0.045058,0.041085,0.003000
5,6,0.045005,0.039143,0.002998
6,7,0.054937,0.041139,0.002002
7,8,0.047000,0.048169,0.003000
8,9,0.056001,0.042047,0.003000
9,10,0.054001,0.043545,0.002000



### Conclusiones:
    1. El tener mayor cantidad de funciónes puede ser mas "ordenado" pero es menos óptimo
    2. Entre mas operaciones entre objetes de una misma librería haga es mejor, pues estos están optimizacos entre sí.
    3. Vectorizar funciónes de ciclos disminuye de manera evidente el tiempo de ejecución
    4. Las librería de modelos de python está muy optimizados las cuales es difícil crear un código con el tiempo de ejución que estos presentan.
    5. Entre más iteraciones se pierde la ambiguedad del tiempo y muestra más claro de cual es más optimo que otro.


### Modificaciones que redujerfon el tiempo:
    1. Primero lo que hice fue reducir el número de métodos externos en el modelo final, no obstante si dejé algunas ya que quitarlas implicaba hacerlo más lento.
    2. La función predict la sustituí completamente por una versión vectorizada y sin for, y también inicializaba variables que estaban dentro de for de estas desde afuera e ingresarla como parámetro para evitar su multiple declaración.
    3. Se quitó el diccionario de return y se devolvió en vector los resultados y los cuales no se definieron anteriormente, sino que en el return.